# Inicializar bibliotecas

In [2]:
!wget https://emanuel3queijos.github.io/AWS-University-ETL/python_scripts/requirements.txt -O requirements.txt
!sed -i '/pywin32/d' requirements.txt
!pip install -r requirements.txt


--2024-11-09 23:39:50--  https://emanuel3queijos.github.io/AWS-University-ETL/python_scripts/requirements.txt
Resolving emanuel3queijos.github.io (emanuel3queijos.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to emanuel3queijos.github.io (emanuel3queijos.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 691 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     691  --.-KB/s    in 0s      

2024-11-09 23:39:51 (32.8 MB/s) - ‘requirements.txt’ saved [691/691]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.

# Scripts para adicionar data dentro do banco



In [3]:
import psycopg2

# Função para criar a conexão
def create_connection():
    try:
        conn = psycopg2.connect(
            host="db-manu-etluniversityproject-instance.c44vb5orqds7.us-east-1.rds.amazonaws.com",
            dbname="AWS-University-ETL-DB",
            user="postgres",
            password="Emanu0710**",
            port=5432
        )

        return conn
    except Exception as error:
        print(f"Erro ao conectar ao banco de dados: {error}")
        return None
create_connection()

<connection object at 0x79f0a8ec3380; dsn: 'user=postgres password=xxx dbname=AWS-University-ETL-DB host=db-manu-etluniversityproject-instance.c44vb5orqds7.us-east-1.rds.amazonaws.com port=5432', closed: 0>

In [5]:
def inserir_departamentos():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            # Inserir departamentos
            departamentos = [
                ("Escola de Tecnologia", "Departamento focado em áreas tecnológicas."),
                ("Escola de Engenharia", "Departamento voltado para as engenharias."),
                ("Escola de Saúde", "Departamento voltado para ciências da saúde."),
                ("Ciências Biológicas", "Departamento voltado para biologia e afins.")
            ]
            for nome, descricao in departamentos:
                cur.execute("""
                    INSERT INTO tb_departamento (nome_departamento, descricao)
                    VALUES (%s, %s)
                    RETURNING cod_departamento
                """, (nome, descricao))
                cod_departamento = cur.fetchone()[0]
                print(f"Departamento {nome} inserido com sucesso. Código: {cod_departamento}")

            conn.commit()
            print("Departamentos e inseridos com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

# Executar o script
inserir_departamentos()


Departamento Escola de Tecnologia inserido com sucesso. Código: 10
Departamento Escola de Engenharia inserido com sucesso. Código: 11
Departamento Escola de Saúde inserido com sucesso. Código: 12
Departamento Ciências Biológicas inserido com sucesso. Código: 13
Departamentos e inseridos com sucesso.


In [7]:
def insert_data():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            # Dicionário onde as chaves são os nomes dos departamentos
            cursos_por_departamento = {
                "Escola de Tecnologia": [
                    ("Engenharia de Software", "Curso voltado ao desenvolvimento de software.", 4, 1200.00),
                    ("Ciência da Computação", "Foco em computação e sistemas.", 4, 1150.00),
                    ("Inteligência Artificial", "Curso para aprendizado de IA.", 4, 1300.00),
                    ("Redes de Computadores", "Infraestrutura e comunicação de redes.", 3, 1100.00)
                ],
                "Escola de Engenharia": [
                    ("Engenharia Civil", "Foco em obras e infraestrutura.", 5, 1400.00),
                    ("Engenharia Elétrica", "Curso de sistemas elétricos.", 5, 1350.00),
                    ("Engenharia Mecânica", "Curso sobre mecânica aplicada.", 5, 1380.00),
                    ("Engenharia de Controle e Automação", "Automação industrial.", 5, 1450.00)
                ],
                "Escola de Saúde": [
                    ("Medicina", "Foco em prática médica.", 6, 2000.00),
                    ("Enfermagem", "Prática de enfermagem.", 4, 1100.00),
                    ("Farmácia", "Estudos sobre medicamentos.", 4, 1250.00),
                    ("Biomedicina", "Ciência biomédica.", 4, 1200.00)
                ],
                "Ciências Biológicas": [
                    ("Biologia", "Estudo da vida.", 4, 1100.00),
                    ("Ecologia", "Curso sobre ecossistemas.", 4, 1050.00),
                    ("Zoologia", "Estudo dos animais.", 4, 1100.00),
                    ("Botânica", "Estudo das plantas.", 4, 1000.00)
                ]
            }

            for nome_departamento, cursos in cursos_por_departamento.items():
                # Busca o código do departamento pelo nome
                cur.execute("""
                    SELECT cod_departamento FROM tb_departamento WHERE nome_departamento = %s
                """, (nome_departamento,))
                cod_departamento = cur.fetchone()

                if cod_departamento is None:
                    print(f"Departamento '{nome_departamento}' não encontrado.")
                    continue

                cod_departamento = cod_departamento[0]

                for nome_curso, descricao_curso, duracao, mensalidade in cursos:
                    cur.execute("""
                        INSERT INTO tb_curso (nome_curso, descricao, cod_departamento, duracao, valor_mensalidade)
                        VALUES (%s, %s, %s, %s, %s)
                        RETURNING cod_curso
                    """, (nome_curso, descricao_curso, cod_departamento, duracao, mensalidade))
                    cod_curso = cur.fetchone()[0]

                    print(f"Curso {nome_curso} inserido com sucesso. Código: {cod_curso}")

            conn.commit()
            print("Departamentos, cursos e disciplinas inseridos com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

# Executar o script
insert_data()


Curso Engenharia de Software inserido com sucesso. Código: 3
Curso Ciência da Computação inserido com sucesso. Código: 4
Curso Inteligência Artificial inserido com sucesso. Código: 5
Curso Redes de Computadores inserido com sucesso. Código: 6
Curso Engenharia Civil inserido com sucesso. Código: 7
Curso Engenharia Elétrica inserido com sucesso. Código: 8
Curso Engenharia Mecânica inserido com sucesso. Código: 9
Curso Engenharia de Controle e Automação inserido com sucesso. Código: 10
Curso Medicina inserido com sucesso. Código: 11
Curso Enfermagem inserido com sucesso. Código: 12
Curso Farmácia inserido com sucesso. Código: 13
Curso Biomedicina inserido com sucesso. Código: 14
Curso Biologia inserido com sucesso. Código: 15
Curso Ecologia inserido com sucesso. Código: 16
Curso Zoologia inserido com sucesso. Código: 17
Curso Botânica inserido com sucesso. Código: 18
Departamentos, cursos e disciplinas inseridos com sucesso.


# operações extras(deleta tudo não vei 😭)

In [ ]:

def export_to_csv():
    conn = create_connection()
    try:
        cur = conn.cursor()

        tables = ["tb_aluno", "tb_curso", "tb_disciplina", "tb_turma", "tb_matricula", "tb_inscricao", "tb_turma_aluno"]
        for table in tables:
            with open(f'{table}.csv', 'w') as f:
                cur.copy_expert(f"COPY {table} TO STDOUT WITH CSV HEADER", f)

        cur.close()
        print("Dados exportados com sucesso.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao exportar dados: {error}")
    finally:
        if conn is not None:
            conn.close()


#Export tables
export_to_csv()

In [ ]:
# Limpar as tabelas
def clean_tables(conn):
    conn = create_connection()
    try:
        cur = conn.cursor()
        truncate_sql = """TRUNCATE TABLE tb_turma_aluno, tb_inscricao, tb_matricula, tb_turma, tb_disciplina, tb_curso, tb_aluno RESTART IDENTITY CASCADE;"""
        cur.execute(truncate_sql)
        conn.commit()
        print("Tabelas limpas com sucesso")
        cur.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao truncar tabelas: {error}")

    finally:
        if conn is not None:
            conn.close()

clean_tables()

Tabelas limpas com sucesso
